Laboratory Task 3-----Bit-error Control

10407315 Haorui Chen  Mar 11th 2019 

# Part 3.1-Effect of increasing transmit power on speech quality 

In [ ]:
! rm pesq_results.txt
import numpy as np
import math
import comp28512_utils as utils
from matplotlib import pyplot 
from scipy.io import wavfile 
from comp28512_utils import get_pesq_scores 
%matplotlib inline

def produceErrAudio(power, freq, arr,y_spe):
    Eb = power*(10**(-5))/(freq*16)
    err = 0.5*math.erfc(math.sqrt(Eb/(10**(-11))))
    print 'Bit-error probability when power=%fwatt'%(err)
    res = utils.insert_bit_errors(arr,err)
    y_res = utils.bit_array_to_numpy_array(res, dtype=y_spe.dtype)
    name = "power=%.4fw.wav"%(power)
    wavfile.write(name, Fs_spe, y_res)
    print name
    utils.audio_from_file(name)
    ! ./linux_pesqmain +8000 NarrobandSpeech8k.wav $name > /dev/null
    pesq_results = get_pesq_scores()
    score = pesq_results["NarrobandSpeech8k.wav"] [name] 
    print "PESQ score when power = %.4fw:"%(power), score

(Fs_spe, y_spe) = wavfile.read("NarrobandSpeech8k.wav"); 
print "original audio without bit-error"
utils.Audio(y_spe, rate=Fs_spe,)    # play original speech
raw = utils.numpy_array_to_bit_array(y_spe)
produceErrAudio(1, Fs_spe, raw, y_spe)
print ""
produceErrAudio(1.35, Fs_spe, raw, y_spe)

dB = []
beP = []
for i in range(0,20):
    dB.append(10*math.log10(10**((i-8)/10.0)))
    beP.append(0.5*math.erfc(math.sqrt(10**((i-8)/10.0))))
fig, ax0 = pyplot.subplots(1)
ax0.set_title("Waterfall graph for msk modulation")
ax0.set_xlabel("Eb over N0 (dB)") ; ax0.set_ylabel("Bit-error probability")
ax0.set_yscale("log")
ax0.plot(dB,beP)
ax0.grid(True, which='major', axis='x')
ax0.grid(True, which='minor', axis='y')

Part 3.1 Q&A:

1. Explain how the appropriate bit-error probability is calculated in Python for a transmission power of 1 watt.  
    -Energy per bit(Eb) is get from 1 watt×(10^(-5))/(sampling freqency×16), where sampling frequency is 8kHz. With Eb, we can get bit-error probability(BeP) by 0.5×erfc(sqrt(Eb/(10^(-12)))).
2. Show how the required graph of bit-error probability against Eb/No (in dB) is produced in Python.  
    -Since Eb/N0 is not in dB, I  and transform x-axis into log scale.
3. What is the bit-error probability when the transmission power is 1 watt?   
    -This is the result of calculation in Q1, that's 3.86×10^(-5) approximately.
4. Explain how the evenly distributed bit-errors are introduced.  
    -In predefined insert_bit_errors, an array with its length equal to that of bit-array await to be altered, approximately (length of array)×(bit-error probility) number of 1's are uniformly distributed across the new array, eventually these two arrays are bitwise xor-ed together.
5. What is the new bit-error probability (for 1.35 watts) and how was this calculated?  
    -Energy per bit(Eb) is get from 1.35 watt×(10^(-5))/(sampling freqency×16), where sampling frequency is 8kHz. With Eb, we can get bit-error probability(BeP) by 0.5×erfc(sqrt(Eb/(10^(-12)))), that's 2.19×10^(-6) approximately.
6. What is the PESQ score for the narrowband speech transmitted with 1.35 watts?  
    -It's 4.29.
7. How is the talk time affected?  
    -Generally, the higher energy per bit, the lower the talk time; If the battery holds 18000 joules, then if we use power of 1watt, we get 18000s talk-time; for power of 1.34watt, we get 13333s.
8. Explain in about one sentence how battery life and bit-error rates are connected.  
    -Since higher energy per bit can avoid bit-error rate more effectively, the lower the bit-error rate, the lower the battery life.

# Part 3.2-Effect ofincreasing bit-error probability on narrow-band speech without FEC scheme

In [ ]:
! rm pesq_results.txt
score_list_2 = []
pos_list = [] 
for i in range(0,13):
    pos = 10**(-5+i/4.0)
    res = utils.insert_bit_errors(raw,pos)
    y_res = utils.bit_array_to_numpy_array(res,y_spe.dtype)
    name = "2_bit_error_prob=%.6f.wav"%(pos)
    wavfile.write(name, Fs_spe, y_res)
    print name
    utils.audio_from_file(name)
    ! ./linux_pesqmain +8000 NarrobandSpeech8k.wav $name > /dev/null
    pesq_results = get_pesq_scores()
    score = pesq_results["NarrobandSpeech8k.wav"] [name] 
    print "PESQ score for possibility = %.6f:"%(pos), score
    print ""
    score_list_2.append(score)
    pos_list.append(pos)
# print score_list
# print pos_list
fig, ax0 = pyplot.subplots(1)
ax0.set_title("PESQ score against beP")
ax0.set_xlabel("Bit-error prob") ; ax0.set_ylabel("PESQ Score")
ax0.plot(pos_list,score_list_2)
ax0.set_xscale("log")
ax0.grid(True, which='minor', axis='x')
ax0.grid(True, which='major', axis='y')

Part 3.2 Q&A:

1. How did you decide how to choose increasing values of bit-error probability? Bearing in mind that your graph should have a logarithmic horizontal scale.  
    -Bit-error grows steadily in logarithmic scale means that power of 10 shall grow steadily starting from -5 to -2 for 12 times, so I decide that bit-error rate 10^(i) is going to have linear growth from -5 to -2 in 12 steps.
2. Comment on the effect of the bit-errors on the sound and how this changes as the bit-error probability increases.  
    -Bit-error adds noises to original audio. As bit-error rate grows, noises start to appear more frequent and influence clearity of dialogue.
3. Do the PESQ scores agree with your perception of the sound?  
    -Yes. Generally, PESQ score will go down as bit-error rate goes up.

# Part 3.3-Effect ofincreasing bit-error probability on narrow-band speech with a (3,1) repetition FEC scheme


In [ ]:
! rm pesq_results.txt
score_list_3 = []
pos_list = []
for i in range(0,13):
    pos = 10**(-5+i/4.0)
    y1 = utils.insert_bit_errors(raw,pos)
    y2 = utils.insert_bit_errors(raw,pos)
    y3 = utils.insert_bit_errors(raw,pos)
    #res = (b2&(b1|b3))|(b1&b3) = (b1&b2)|(b1&b3)|(b2&b3)
    res = np.bitwise_or(np.bitwise_and(y2,np.bitwise_or(y1,y3)),np.bitwise_and(y1,y3))
    y_res = utils.bit_array_to_numpy_array(res,y_spe.dtype)
    name = "3_bit_error_prob=%.6f.wav"%(pos)
    wavfile.write(name, Fs_spe, y_res)
    print name
    utils.audio_from_file(name)
    ! ./linux_pesqmain +8000 NarrobandSpeech8k.wav $name > /dev/null
    pesq_results = get_pesq_scores()
    score = pesq_results["NarrobandSpeech8k.wav"] [name] 
    print "PESQ score for possibility = %.6f:"%(pos), score
    print ""
    score_list_3.append(score)
    pos_list.append(pos)
# print score_list
# print pos_list
fig, ax0 = pyplot.subplots(1)
ax0.set_title("PESQ score against beP")
ax0.set_xlabel("Bit-error prob") ; ax0.set_ylabel("PESQ Score")
ax0.plot(pos_list,score_list_3, color='green', label='voting')
ax0.plot(pos_list,score_list_2, color='skyblue', label='no voting')
ax0.set_xscale("log")
ax0.legend()
ax0.grid(True, which='minor', axis='x')
ax0.grid(True, which='major', axis='y')

# plt.title('PESQ score against beP')
# plt.plot(pos_list, score_list_3, color='green', label='voting')
# plt.plot(pos_list, score_list_2, color='skyblue', label='autocracy')
# plt.legend() # 显示图例

# plt.xlabel('Bit-error prob'); plt.ylabel('PESQ Score')
# plt.show()

Part 3.3 Q&A:
    
1. Demonstrate and comment on any improvement in the narrow-band speech quality that is obtained at the expense of the 3-fold increase in the bit-rate. How would you summarise the degree of improvement?  
    -When bit-error rate < 10^(-3), improvement on narrow-band speech quality under 3-fold coding is obvious, I can hear almost no other sounds apart from dialogue and improvement in PESQ score is also considerable. For bit-rate larger than 10^(-3), noise gradually appear, yet audio quality is still better than that without processing under 3-fold. 
2. What is the effect of the 3-fold increase in bit-rate on the transmit power and on the energy required to transmit the speech segment?  
    -Total bits in file remain unchanged and power per bit is stable, but as we are sending same information 3 times, energy required is also 3 times than before.
3. If we reduce the energy per bit by a factor of 3, the transmit power becomes what it was before? Consider the example in Section 2.5 where Eb/N0 was 8.93 dB. It would now become 8.93–  10xlog10(3)=8.93-4.8 dB=4.3dB. From the msk waterfall graph above, the beP now becomes about 0.01. How well does the (3,1) rep coder work at beP=0.01?  
    -It becomes 1/3 of previous energy. When beP=0.01, PESQ before 3-fold is 1.453, after is 1.878. Obviously it gives better result than that when 3-fold coder is not applied at all, yet there are still niticable noices, thus the result is unaccepted.
4. What other method could be used to reduce the energy required to transmit the speech sentence without reducing the energy per bit? Think about previous Tasks.  
    -We can reduce bit-rate(i.e. by resampling), number of bit per sample or apply Huffman coding.

# Part 3.4-Apply ARQ to a text message

In [ ]:
import operator
def checkCRC8(xa) :  
   # Generates 8-bit CRC check as 1x8 row-vector
   # xa is array of bits expressed as a row-vector
   # Generator polynomial is g(x) = x^8 + x^2 +x + 1
    g8x = [1,0,0,0,0,0,1,1,1] ;  # Generator polynomial as row-vector
    xae = np.append(xa, [0,0,0,0,0,0,0,0] );  # Append 8 zeros
    xsa=xae[0:9] #Get 1st 9bits from 0 to 8
    for i in range (0 , len(xa) ) : #loop for all the bits in original array
        if xsa[0] == g8x[0]: #If 1st bit is a 1, perform xor
            xsa = xsa ^ g8x  #result of xor
        xsa[0:8] = xsa[1:9]  #Shift the result one place left
        if ( i < len(xa)-1 ) : #Make sure we are not at end of xoring part of CRC algorithm (-1 because arrays start at 0)
            xsa[8] = xae[ i + 9] #Pull the next bit of the array down and put it in right most bit of result
    check = xsa[0:8] #Get reminder   
    return check #return the CRC value

raw = 'Smart-phones are mobile games consoles and mp3 players that you can also use for telephone calls.'
bit_raw = utils.bytes_to_bit_array(raw)
rem = checkCRC8(bit_raw)
print 'original CRC:',rem
bit_raw = np.append(bit_raw,rem)
for i in range(0,13):
    print ''
    pos = 10**(-4+i/4.0)
    print 'beP=%.5f'%(pos)
    #transmit
    for i in range(0,9):
        alt = utils.insert_bit_errors(bit_raw,pos)
        rlt = checkCRC8(alt)
        print 'CRC after transmission:', rlt
        print utils.bit_array_to_bytes(alt[0:len(alt)-8])
        if(np.array_equal(rlt,[0,0,0,0,0,0,0,0])):
            print 'success'
            print ''
            break
        else:
            print 'fail'

Part 3.4 Q&A:
    
1. If the CRC8 check succeeds, can we deduce that there are no bit-errors?   
    -No. Any combination of bit-errors that ‘adds’ any ‘multiple’ of given CRC will not be detected.
2. If the CRC8 check fails, can we deduce that there are some bit-errors?      
    -Yes. When doing CRC, change of bits on dividend is likely to change reaminder, so if remainder changed after introducing bit-errors, it means that bit-errors must have occured.
3. Explain how your results demonstrate the effect of evenly distributed bit-errors and ARQ with various values of bit-error probability.  
    -For bit errors, we can see that throughout the growth of bit-error probability, wrong characters in result string always appear evenly across it and get more and more dense but not in clusters, thus are evenly distributed; for ARQ, it detects whether reaminder remains same as that before introducing bit-error, if they are different then a 'fail' is reported and we know ARQ is working.
4. For what values of bit-error probability did the ARQ process fail completely?  
    -When bit-error at 0.00562.

# Part 3.5-Applying a modified form of ARQ to a text message

In [ ]:
for i in range(0,13):
    pos = 10**(-4+i/4.0)
    print 'beP=%.5f'%(pos)
    #transmit
    for j in range(0,3):
        alt = utils.insert_bit_errors(bit_raw,pos)
#         res=alt[0:len(alt)-8]
        rlt = checkCRC8(alt)
        print 'CRC after transmission:', rlt
        print utils.bit_array_to_bytes(alt[0:len(alt)-8])
        if(np.array_equal(rlt,[0,0,0,0,0,0,0,0])):
            print 'success'
            print ''
            break
        else:
            print 'fail'
            if j==0:
                y1 = alt
            elif j==1:
                y2 = alt
            else:
                y3 = alt
                #res = (b2&(b1|b3))|(b1&b3) = (b1&b2)|(b1&b3)|(b2&b3)
                res = np.bitwise_or(np.bitwise_and(y2,np.bitwise_or(y1,y3)),np.bitwise_and(y1,y3))
#                 fin = checkCRC8(np.append(res,[0,0,0,0,0,0,0,0]))
                fin = checkCRC8(res)
                print fin
                if(np.array_equal(fin,[0,0,0,0,0,0,0,0])):
                    print 'Voting success'
                else:
                    print 'Voting failed'
                print 'Result of chase combining:',utils.bit_array_to_bytes(res[0:len(res)-8])
    print''

Part 3.5 Q&A:
    
1. What could the ARQ mechanism above do if a numerical or string array is received correctly but bit-errors have occurred in the CRC8 check-bits?  How often might this occur?   
    -It will require a retransmission. It is likely to happen often because bit-errors tend to be evenly distributed.
2. State whether you consider ARQ as used in Part 3.4 an efficient method, and explain the reasons for your answer.  
    -No, because we haven't take advantage of previous failed trials, and for every time we tend to have a same probability of failing without improvement, thus the method is unefficient.
3. Explain your mechanism for combining failed retransmissions.  
    -I will store 3 failed retransmissions and execute a ‘majority voting’ procedure. For each bit in 3 wrong string, if no less than 2 of 3 version on that bit are the same, then I believe the majority result should be the one in final result. For implementation, if we represent 3 version of current bit as b1, b2, b3, then we get result bit by calculating (b1&b2)|(b1&b3)|(b2&b3).
4. Demonstrate the effect of combining 3 failed transmissions.  
    -Generally, the result of combination looks much closer to original string than 3 failed transmissions, which is the result in 3.4, yet when a voting needs to occur, errors in all 3 versions of array are all considerable, thus the final version is likely not to be same as original version. 
5. At what value of beP does the new ARQ process fail completely? Is there any improvement over what we got before?  
    -At beP=0.03162, the new ARQ process is failing completely. There indeed are improvements, yet the result still has obvious errors.
6. If combining failed transmissions turns out to be a good idea, why is it not used in practice, for example in wifi (you need to look this up).  
    -With chase-combining we need to allocate extra space for multiple versions of received message(reference doc: ieee 802.11-18/1955r0). 